<a href="https://colab.research.google.com/github/ehdwls314/prac_class/blob/main/%EB%8D%B0%EB%A7%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb


In [21]:
import keras

In [3]:
total = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [4]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


In [5]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [6]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [7]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [8]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [9]:
total_A = total[total['구분']==0]
total_B = total[total['구분']==1]
total_C = total[total['구분']==2]
total_D = total[total['구분']==3]
total_E = total[total['구분']==4]
total_G = total[total['구분']==5]
total_H = total[total['구분']==6]

In [10]:
weather = pd.read_csv('data_with_weather.csv')

In [11]:
weather.head()

,연월일,시간,구분,공급량,기온
0,2013-01-01,1,A,2497.129,-6.609783
1,2013-01-01,2,A,2363.265,-7.004348
2,2013-01-01,3,A,2258.505,-7.214130
3,2013-01-01,4,A,2243.969,-7.300000
4,2013-01-01,5,A,2344.105,-7.511957


In [12]:
weather = weather['기온']


In [13]:
total = pd.concat([total,weather],axis=1)

In [14]:
total.head()

,연월일,시간,구분,공급량,year,month,day,weekday,기온
0,2013-01-01,1,0,2497.129,2013,1,1,1,-6.609783
1,2013-01-01,2,0,2363.265,2013,1,1,1,-7.004348
2,2013-01-01,3,0,2258.505,2013,1,1,1,-7.214130
3,2013-01-01,4,0,2243.969,2013,1,1,1,-7.300000
4,2013-01-01,5,0,2344.105,2013,1,1,1,-7.511957


In [15]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [16]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [17]:
features = ['구분', 'month', 'day', 'weekday', '시간', '기온']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']


In [18]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'task' : 'train',
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 262.82
[40]	valid_0's l1: 194.409
[60]	valid_0's l1: 175.812
[80]	valid_0's l1: 169.478
[100]	valid_0's l1: 166.155
[120]	valid_0's l1: 165.067
Early stopping, best iteration is:
[116]	valid_0's l1: 164.628


In [29]:
model.predict(train_x, train_y)

AttributeError: ignored

In [ ]:
history_size = 30*24*3
target_size = 2160

y_A = y_A.loc[y_A.date >= datetime(2016, 1, 1)]
weather = weather[-len(y_A):]

y_A = pd.concat([y_A[['supply']], weather[['체감온도']]], axis=1)

X_train, y_train = prepare_data(y_A, history_size, target_size)


In [26]:
history_size = 30*24*3
target_size = 2160


In [27]:
x_test = []
x_test.append(total_A.values[-history_size:])
x_test = np.array(x_test)
x_test.shape

(1, 2160, 8)

In [28]:
model.fit(train_x, train_y)

AttributeError: ignored

In [83]:
best_lgbm = get_best_params(model, params)

NameError: ignored

In [12]:
train.groupby(['month'])['공급량'].mean()

month
1     1791.931346
2     1622.932402
3     1210.662557
4      799.846869
5      542.799101
6      474.749196
7      471.275057
8      444.279738
9      460.747752
10     611.115468
11    1137.585059
12    1684.639508
Name: 공급량, dtype: float64

In [13]:
train.groupby(['구분'])['공급량'].mean()

구분
0    1157.195134
1     949.208549
2     132.983272
3     656.975075
4    1372.319323
5    2028.477825
6     246.889076
Name: 공급량, dtype: float64

In [ ]:
# 원핫 인코딩 (pd.get_dummies())

train_one = pd.get_dummies(train)

test_one = pd.get_dummies(test)



# 모델 정의

model = LGBMClassifier()



# 모델 학습



# X 는 train에서 quality 를 제외한 모든 변수

# y 는 train의 qulity 변수



X = train_one.drop('quality',axis= 1)

y = train_one['quality']



# fit 메소드를 이용해 모델 학습

model.fit(X,y)



# predict 메소드와 test_one 데이터를 이용해 품질 예측

pred = model.predict(test_one)



# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기

submission = pd.read_csv('data/sample_submission.csv')

submission['quality'] = pred

submission.head()

In [ ]:
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')